In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib
import re


In [12]:
rpm_df = pd.read_csv('/kaggle/input/RPM_dataset.csv')

rpm_df.columns = ['Timestamp', 'CAN ID', 'DLC', 'DATA0', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5', 'DATA6', 'DATA7', 'Flag']
rpm_df.head()

,Timestamp,CAN ID,DLC,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
0,1.478191e+09,018f,8,fe,3b,00,00,00,3c,00,00,R
1,1.478191e+09,0260,8,19,22,22,30,ff,8f,6e,3f,R
2,1.478191e+09,02a0,8,60,00,83,1d,96,02,bd,00,R
3,1.478191e+09,0329,8,dc,b8,7e,14,11,20,00,14,R
4,1.478191e+09,0545,8,d8,00,00,83,00,00,00,00,R


In [13]:
rpm_df.nunique()

Timestamp    4621701
CAN ID            26
DLC                2
DATA0            113
DATA1             85
DATA2             89
DATA3             28
DATA4            192
DATA5            256
DATA6             80
DATA7            256
Flag               2
dtype: int64

In [14]:
df_2 = rpm_df.copy()

In [15]:
missing_values = df_2.isnull().sum()
missing_values

Timestamp        0
CAN ID           0
DLC              0
DATA0            0
DATA1            0
DATA2            0
DATA3        41476
DATA4        41476
DATA5        41476
DATA6        41476
DATA7        41476
Flag         41476
dtype: int64

In [16]:
df_dlc_is_2 = df_2[df_2["DLC"] == 2].copy()
df_dlc_is_2.head(), df_dlc_is_2.shape

(        Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
 42   1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 134  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 227  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 319  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 412  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 
     Flag  
 42   NaN  
 134  NaN  
 227  NaN  
 319  NaN  
 412  NaN  ,
 (41476, 12))

In [17]:
df_d2_nan = df_2[df_2["DATA2"] == 'R'].copy()
df_d2_nan.head(), df_d2_nan.shape

(        Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
 42   1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 134  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 227  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 319  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 412  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 
     Flag  
 42   NaN  
 134  NaN  
 227  NaN  
 319  NaN  
 412  NaN  ,
 (41476, 12))

In [18]:
df_flag_nan = df_2[(df_2["Flag"] != "T") & (df_2["Flag"] != "R")]
df_flag_nan.head(10), df_flag_nan.shape

(        Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
 42   1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 134  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 227  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 319  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 412  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 504  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 597  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 689  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 782  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 874  1.478191e+09   05f0    2    01    00     R   NaN   NaN   NaN   NaN   NaN   
 
     Flag  
 42   NaN  
 134  NaN  
 227  NaN  
 319  NaN  
 412  NaN  
 504  NaN  
 597  NaN  


In [19]:
df_3 = df_2.copy()

In [20]:
# For rows with DLC=2, move 'R' from DATA2 to Flag
mask = df_3["DLC"] == 2
df_3.loc[mask, "Flag"] = df_3.loc[mask, "DATA2"]  # Copy 'R' to Flag
df_3.loc[mask, "DATA2"] = np.nan  # Set DATA2 to NaN for DLC=2



# Verify alignment
print(df_3[df_3["DLC"] == 2].head())

        Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
42   1.478191e+09   05f0    2    01    00   NaN   NaN   NaN   NaN   NaN   NaN   
134  1.478191e+09   05f0    2    01    00   NaN   NaN   NaN   NaN   NaN   NaN   
227  1.478191e+09   05f0    2    01    00   NaN   NaN   NaN   NaN   NaN   NaN   
319  1.478191e+09   05f0    2    01    00   NaN   NaN   NaN   NaN   NaN   NaN   
412  1.478191e+09   05f0    2    01    00   NaN   NaN   NaN   NaN   NaN   NaN   

    Flag  
42     R  
134    R  
227    R  
319    R  
412    R  


In [21]:
df_4 = df_3.copy()

In [22]:
# Fill NaN with hex 00
default_hex = '00'
data_columns = ['DATA2', 'DATA3', 'DATA4', 'DATA5', 'DATA6', 'DATA7', 'Flag']
df_4[data_columns] = df_4[data_columns].fillna(default_hex)
print(df_4[df_4["DLC"] == 2].head()), df_4[df_4["DLC"] == 2].shape

        Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
42   1.478191e+09   05f0    2    01    00    00    00    00    00    00    00   
134  1.478191e+09   05f0    2    01    00    00    00    00    00    00    00   
227  1.478191e+09   05f0    2    01    00    00    00    00    00    00    00   
319  1.478191e+09   05f0    2    01    00    00    00    00    00    00    00   
412  1.478191e+09   05f0    2    01    00    00    00    00    00    00    00   

    Flag  
42     R  
134    R  
227    R  
319    R  
412    R  


(None, (41476, 12))

In [23]:
df_4.head(), df_4.shape

(      Timestamp CAN ID  DLC DATA0 DATA1 DATA2 DATA3 DATA4 DATA5 DATA6 DATA7  \
 0  1.478191e+09   018f    8    fe    3b    00    00    00    3c    00    00   
 1  1.478191e+09   0260    8    19    22    22    30    ff    8f    6e    3f   
 2  1.478191e+09   02a0    8    60    00    83    1d    96    02    bd    00   
 3  1.478191e+09   0329    8    dc    b8    7e    14    11    20    00    14   
 4  1.478191e+09   0545    8    d8    00    00    83    00    00    00    00   
 
   Flag  
 0    R  
 1    R  
 2    R  
 3    R  
 4    R  ,
 (4621701, 12))

In [24]:
df_4[(df_4["Flag"] != "T") & (df_4["Flag"] != "R")]


,Timestamp,CAN ID,DLC,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag


In [25]:
df_4[df_4["DATA2"] == 'R'].head()

,Timestamp,CAN ID,DLC,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag


In [26]:
# Drop unnecessary columns
df_drop_dlc = df_4.drop(["DLC"], axis=1).copy()
df_drop_dlc.head()

,Timestamp,CAN ID,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
0,1.478191e+09,018f,fe,3b,00,00,00,3c,00,00,R
1,1.478191e+09,0260,19,22,22,30,ff,8f,6e,3f,R
2,1.478191e+09,02a0,60,00,83,1d,96,02,bd,00,R
3,1.478191e+09,0329,dc,b8,7e,14,11,20,00,14,R
4,1.478191e+09,0545,d8,00,00,83,00,00,00,00,R


In [27]:
df_drop_dlc.dtypes, df_drop_dlc.shape

(Timestamp    float64
 CAN ID        object
 DATA0         object
 DATA1         object
 DATA2         object
 DATA3         object
 DATA4         object
 DATA5         object
 DATA6         object
 DATA7         object
 Flag          object
 dtype: object,
 (4621701, 11))

In [28]:
df_can_id_0000 = df_drop_dlc[df_drop_dlc["CAN ID"] == "0000"]
df_can_id_0000.head(), df_can_id_0000.shape

(Empty DataFrame
 Columns: [Timestamp, CAN ID, DATA0, DATA1, DATA2, DATA3, DATA4, DATA5, DATA6, DATA7, Flag]
 Index: [],
 (0, 11))

In [29]:
df_6 = df_drop_dlc.copy()

df_drop_timestamp = df_drop_dlc.drop(columns=['Timestamp'], inplace=False).copy()
df_drop_timestamp.describe()



,CAN ID,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
count,4621701,4621701,4621701,4621701,4621701,4621701,4621701,4621701,4621701,4621701
unique,26,113,85,88,28,192,256,80,256,2
top,0316,00,00,00,00,00,00,00,00,R
freq,871230,1467124,1710232,2406008,1888765,1790965,1298354,2822119,2131776,3966804


In [32]:
data_columns = ['DATA0', 'DATA1', 'DATA2', 'DATA3', 'DATA4', 'DATA5', 'DATA6', 'DATA7']

# Create regex pattern for valid hex
hex_pattern = r'^[0-9A-Fa-f]{2}$'

# Check for non-hex values
mask = df_6[data_columns].apply(lambda col: ~col.str.match(hex_pattern, na=False))

# Get rows with any invalid entries
invalid_rows = df_6[mask.any(axis=1)]

print("Rows with non-hex values in DATA columns:")
print(invalid_rows if not invalid_rows.empty else "No non-hex values found")

Rows with non-hex values in DATA columns:
No non-hex values found


In [33]:
df_7 = df_6.copy()
# Function to convert hex to decimal
def hex_to_int(hex_str: str) -> int:
    try:
        return int(str(hex_str).strip(), 16)  # Convert hex to int
    except ValueError:
        return np.nan 

# Convert all DATA columns
for col in df_7.columns[1:-1]:  # Exclude 'Flag' column
    df_7[col] = df_7[col].apply(hex_to_int)

df_7.head()

,Timestamp,CAN ID,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
0,1.478191e+09,399,254,59,0,0,0,60,0,0,R
1,1.478191e+09,608,25,34,34,48,255,143,110,63,R
2,1.478191e+09,672,96,0,131,29,150,2,189,0,R
3,1.478191e+09,809,220,184,126,20,17,32,0,20,R
4,1.478191e+09,1349,216,0,0,131,0,0,0,0,R


In [34]:
df_8 = df_7.copy()

In [35]:
df_8["Flag"] = df_8["Flag"].map({"R": 0, "T": 1})
df_8.head()

,Timestamp,CAN ID,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
0,1.478191e+09,399,254,59,0,0,0,60,0,0,0
1,1.478191e+09,608,25,34,34,48,255,143,110,63,0
2,1.478191e+09,672,96,0,131,29,150,2,189,0,0
3,1.478191e+09,809,220,184,126,20,17,32,0,20,0
4,1.478191e+09,1349,216,0,0,131,0,0,0,0,0


In [36]:
df_8["Flag"].unique()

array([0, 1])

In [37]:
df_8.isna().sum()

Timestamp    0
CAN ID       0
DATA0        0
DATA1        0
DATA2        0
DATA3        0
DATA4        0
DATA5        0
DATA6        0
DATA7        0
Flag         0
dtype: int64

In [38]:
df_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4621701 entries, 0 to 4621700
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Timestamp  float64
 1   CAN ID     int64  
 2   DATA0      int64  
 3   DATA1      int64  
 4   DATA2      int64  
 5   DATA3      int64  
 6   DATA4      int64  
 7   DATA5      int64  
 8   DATA6      int64  
 9   DATA7      int64  
 10  Flag       int64  
dtypes: float64(1), int64(10)
memory usage: 387.9 MB


In [39]:
df_8.describe()

,Timestamp,CAN ID,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag
count,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06,4.621701e+06
mean,1.478194e+09,7.151190e+02,6.028397e+01,4.256768e+01,3.530757e+01,9.074923e+01,5.346931e+01,6.175567e+01,2.246208e+01,7.470137e+01,1.417004e-01
std,3.565417e+03,3.505162e+02,8.686002e+01,4.973679e+01,5.134037e+01,1.092712e+02,7.584148e+01,7.354858e+01,5.328198e+01,9.750145e+01,3.487427e-01
min,1.478191e+09,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.478192e+09,3.990000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.478192e+09,7.900000e+02,1.600000e+01,3.300000e+01,0.000000e+00,2.000000e+01,2.200000e+01,3.600000e+01,0.000000e+00,1.900000e+01,0.000000e+00
75%,1.478193e+09,8.800000e+02,6.900000e+01,4.100000e+01,3.600000e+01,2.550000e+02,5.900000e+01,1.270000e+02,8.000000e+00,1.420000e+02,0.000000e+00
max,1.478201e+09,1.680000e+03,2.550000e+02,1.920000e+02,2.520000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.090000e+02,2.550000e+02,1.000000e+00


In [ ]:
# df_8.to_csv('/kaggle/working/cleaned_data.csv', index=False)

# print("Data saved successfully as 'cleaned_data.csv'.")

In [ ]:
# Exclude the 'Flag' column before calculating correlation
correlation_matrix = df_8.drop(columns=['Timestamp', 'CAN ID', 'Flag']).corr()
correlation_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'correlation_matrix' is already computed
plt.figure(figsize=(8, 6))

# Plot the correlation matrix as a heatmap
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', linewidths=0.5)

# Save the plot as a PNG image
plt.tight_layout()  # Ensures that the layout doesn't get cropped
plt.savefig('correlation_matrix_heatmap.png', dpi=300, bbox_inches='tight')

# Optionally, display the plot (if needed)
# plt.show()


In [ ]:
df_9 = df_8.copy()
df_9.head()

# Realistic Scenario with no CAN ID included

## Algorithm Selection XGB

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Load data
df = df_9.drop(columns=['Timestamp']).copy()

# 1. Preprocessing
# Convert CAN ID to numerical (already done as 0 for attacks)
df['CAN ID'] = df['CAN ID'].astype('int64')

# 2. Define features and target variable
X = df.drop(['Flag', 'CAN ID'], axis=1)  # Remove the leakage feature
y = df['Flag']

# 3. Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

# 4. Initialize XGBoost with realistic parameters
model = xgb.XGBClassifier(
    scale_pos_weight=8,
    objective='binary:logistic',
    eval_metric='aucpr',
    use_label_encoder=False,
    max_depth=6,  # Prevent overfitting
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 5. Train the model
model.fit(X_train, y_train)

# 6. Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# 7. Feature Importance
plt.figure(figsize=(10, 6))
plt.title("Feature Importances (Without CAN ID Leakage)")
xgb.plot_importance(model, ax=plt.gca())
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# 1. Plot Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'max_depth': [3, 6, 9],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [50, 100, 200]
# }

# grid_search = GridSearchCV(xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
#                            param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
# grid_search.fit(X_train, y_train)
# print("Best parameters:", grid_search.best_params_)

##  Advanced Feature Engineering

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve
import xgboost as xgb

# Load dataset
df = df_9.copy()
print(df.columns)


# Define feature columns and target
features = ['DATA0', 'DATA1', 'DATA5', 'DATA7']
X = df[features]
y = df['Flag']  # Target: 1 = Attack, 0 = Normal

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Probability scores

# Performance Metrics
print(classification_report(y_test, y_pred))
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")

# Precision-Recall Curve & Optimal Threshold
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
target_recall = 0.95
idx = np.argmax(recall >= target_recall)
optimal_threshold = thresholds[idx]
print(f"Optimal Decision Threshold: {optimal_threshold:.2f}")

best_idx = np.argmax(precision * recall)  # F1-score based tuning
best_threshold = thresholds[best_idx]
print(f"New Optimal Threshold: {best_threshold:.2f}")


# Feature Importance Analysis
# plt.figure(figsize=(8, 5))
# xgb.plot_importance(model, importance_type='weight', title="Feature Importance")
# plt.show()

# Byte-Level Analysis
plt.figure(figsize=(12, 6))
for i, feature in enumerate(features[:2]):  # DATA0 and DATA1
    plt.subplot(1, 2, i + 1)
    sns.histplot(df[df['Flag'] == 0][feature], label='Normal', color='blue', kde=True, alpha=0.6)
    sns.histplot(df[df['Flag'] == 1][feature], label='Attack', color='red', kde=True, alpha=0.6)
    plt.title(f"Distribution of {feature}")
    plt.legend()

plt.show()

# Feature Engineering (Bitwise & Arithmetic Transformations)
df['DATA01 XOR'] = df['DATA0'] ^ df['DATA1']
df['DATA01 SUM'] = df['DATA0'] + df['DATA1']
df['DATA57 XOR'] = df['DATA5'] ^ df['DATA7']

print(df[['DATA01 XOR', 'DATA01 SUM', 'DATA57 XOR']].describe()) 

# Train again with engineered features
X_new = df[['DATA0', 'DATA1', 'DATA5', 'DATA7', 'DATA01 XOR', 'DATA01 SUM', 'DATA57 XOR']]
X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)
model.fit(X_train_new, y_train)

# Evaluate again
y_pred_new = model.predict(X_test_new)
print(classification_report(y_test, y_pred_new))


In [ ]:
df_9.head()

## Temporal Features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE  # Import for oversampling
from sklearn.metrics import roc_auc_score

# Load data
df = df_9.copy()

# Ensure Timestamp is in datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')  # Adjust unit if needed

# Sort values by Timestamp and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

# Set 'Timestamp' as the index for rolling operations
df.set_index('Timestamp', inplace=True)

# 2. Global time-based features (no ECU grouping)
df['Global_Time_Delta'] = df.index.to_series().diff().dt.total_seconds().fillna(0)  # Time between ALL messages

# 3. Rolling count of messages in 1s and 5s window
df['Rolling_Count_DoS_1ms'] = df.rolling('1ms').count()['CAN ID']
df['Rolling_Count_DoS_10ms'] = df.rolling('10ms').count()['CAN ID']

# 4. Frequency-based features (DoS Attack detection)
df['Rolling_Count_DoS_1ms'] = df.rolling('0.3ms').count()['CAN ID']  # Messages in 0.3ms window (for DoS attack)

# 5. Frequency/Time features
df['Hour'] = df.index.hour
df['Time_Since_First_Message'] = (df.index - df.index.min()).total_seconds()

# 6. Prepare features (drop the identifier columns but don't drop 'Timestamp' since it's the index)
X = df.drop(['Flag', 'CAN ID'], axis=1)  # Keep 'Is_DoS_Attack'
y = df['Flag']

# 1. Apply SMOTE for oversampling to handle class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Initialize XGBoost with realistic parameters and scale_pos_weight
model = xgb.XGBClassifier(
    scale_pos_weight=10,  # Adjusting to 10
    objective='binary:logistic',
    eval_metric='aucpr',
    use_label_encoder=False,
    max_depth=6,  # Prevent overfitting
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 2. Initialize StratifiedKFold for 5-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3. Cross-validation loop
roc_auc_scores = []
accuracies = []
classification_reports = []
confusion_matrices = []

for train_index, test_index in kf.split(X_res, y_res):
    X_train, X_test = X_res.iloc[train_index], X_res.iloc[test_index]
    y_train, y_test = y_res.iloc[train_index], y_res.iloc[test_index]

    # Train the model on resampled data
    model.fit(X_train, y_train)

    # Predict and evaluate the model
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Check shapes
    print(f"Shape of y_test: {y_test.shape}")
    print(f"Shape of y_pred: {y_pred.shape}")

    # Calculate metrics
    accuracies.append(accuracy_score(y_test, y_pred))
    roc_auc_scores.append(roc_auc_score(y_test, y_proba))
    classification_reports.append(classification_report(y_test, y_pred))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))

# Print average results after cross-validation
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average ROC AUC Score: {np.mean(roc_auc_scores):.4f}")
print(f"Unique values in y_res: {np.unique(y_res)}")


# Confusion Matrix - Average of all 5 folds
avg_conf_matrix = np.mean(confusion_matrices, axis=0)
print(f"\nAverage Confusion Matrix:\n {avg_conf_matrix}")

# Classification Report - Average of all 5 folds (not directly average-able, but print for one fold)
print("\nClassification Report (example of one fold):\n", classification_reports[0])

# 4. Feature Importance
# Get feature importances
feature_importances = model.feature_importances_

# Extract feature importances from the model
importances = model.get_booster().get_score(importance_type='gain')  # Get 'gain' importance

# Normalize the importance values to make them sum to 1
normalized_importances = {feature: imp / sum(importances.values()) for feature, imp in importances.items()}

# Create a DataFrame for the feature importances
feature_importance_df = pd.DataFrame(list(normalized_importances.items()), columns=['Feature', 'Importance'])

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 1. Plot Feature Importance using XGBoost's internal plot (using normalized 'gain')
plt.figure(figsize=(10, 6))
plt.title("Feature Importances (Including DoS Attack Detection)")
xgb.plot_importance(model, importance_type='gain', ax=plt.gca())  # Use 'gain' for consistency
plt.show()

# 2. Plot Feature Importance using Heatmap for better visualization (Normalized values)
plt.figure(figsize=(8, 6))
sns.heatmap(feature_importance_df.set_index('Feature').T, annot=True, cmap='coolwarm', cbar=False, fmt='.3f', annot_kws={'size': 12})
plt.title('Feature Importance Table')
plt.tight_layout()
plt.show()

# Print the feature importance table
print("\nFeature Importances (Normalized):\n", feature_importance_df)


# Plot ROC Curve for each fold
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc_score(y_test, y_proba):.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve - Fold {len(roc_auc_scores)}')
plt.legend(loc='lower right')
plt.show()

# Plot Confusion Matrix for each fold
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Normal', 'DoS Attack'], yticklabels=['Normal', 'DoS Attack'])
plt.title(f'Confusion Matrix - Fold {len(confusion_matrices)}')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


# #Hyperparameter Tuning using GridSearchCV
# from sklearn.model_selection import GridSearchCV
# from imblearn.pipeline import Pipeline  # To avoid data leakage

# # Define pipeline with SMOTE and XGBoost
# pipeline = Pipeline([
#     ('smote', SMOTE(sampling_strategy='auto', random_state=42)),  # Oversampling
#     ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric="aucpr"))
# ])

# # Define parameter grid
# param_grid = {
#     'xgb__max_depth': [4, 6, 8],  
#     'xgb__learning_rate': [0.01, 0.1],  
#     'xgb__subsample': [0.8, 1.0]
# }

# # Perform Grid Search
# grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=3, n_jobs=-1, verbose=2)
# grid_search.fit(X, y)  # Use original X, y (not resampled)

# # Print best parameters
# print("Best parameters:", grid_search.best_params_)



## Code with GridSearch

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, StratifiedKFold
# import xgboost as xgb
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# from imblearn.over_sampling import SMOTE  # Import for oversampling
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import GridSearchCV, StratifiedKFold
# from imblearn.pipeline import Pipeline  # Use imblearn's Pipeline for SMOTE integration

# # Load data
# df = df_9.copy()

# # Ensure Timestamp is in datetime format
# df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')  # Adjust unit if needed

# # Sort values by Timestamp and reset the index
# df = df.sort_values('Timestamp').reset_index(drop=True)

# # Set 'Timestamp' as the index for rolling operations
# df.set_index('Timestamp', inplace=True)

# # 2. Global time-based features (no ECU grouping)
# df['Global_Time_Delta'] = df.index.to_series().diff().dt.total_seconds().fillna(0)  # Time between ALL messages

# # 3. Rolling count of messages in 1s and 5s window
# df['Rolling_Count_DoS_1ms'] = df.rolling('1ms').count()['CAN ID']
# df['Rolling_Count_DoS_10ms'] = df.rolling('10ms').count()['CAN ID']

# # 4. Frequency-based features (DoS Attack detection)
# df['Rolling_Count_DoS_1ms'] = df.rolling('0.3ms').count()['CAN ID']  # Messages in 0.3ms window (for DoS attack)

# # 5. Frequency/Time features
# df['Hour'] = df.index.hour
# df['Time_Since_First_Message'] = (df.index - df.index.min()).total_seconds()

# # 6. Prepare features (drop the identifier columns but don't drop 'Timestamp' since it's the index)


# # Load data and prepare features (same as before)
# df = df_9.copy()
# df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
# df = df.sort_values('Timestamp').reset_index(drop=True)
# df.set_index('Timestamp', inplace=True)

# # ... [Your existing feature engineering code] ...

# # Prepare X and y (do NOT apply SMOTE here)
# X = df.drop(['Flag', 'CAN ID'], axis=1)
# y = df['Flag']

# # Define the pipeline: SMOTE -> XGBoost
# pipeline = Pipeline([
#     ('smote', SMOTE(random_state=42)),
#     ('xgb', xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='aucpr',
#         use_label_encoder=False,
#         random_state=42
#     ))
# ])

# # Define hyperparameter grid
# param_grid = {
#     'xgb__max_depth': [4, 6],  # Test shallow and deeper trees
#     'xgb__learning_rate': [0.01, 0.1],  # Step size for boosting
#     'xgb__n_estimators': [100, 200],  # Number of trees
#     'xgb__subsample': [0.8, 1.0],  # Fraction of samples per tree
#     'xgb__colsample_bytree': [0.8, 1.0],  # Fraction of features per tree
#     'xgb__scale_pos_weight': [1, 5, 10]  # Handle class imbalance
# }

# # Configure GridSearchCV
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# grid_search = GridSearchCV(
#     estimator=pipeline,
#     param_grid=param_grid,
#     scoring='roc_auc',  # Prioritize AUC for class imbalance
#     cv=kf,
#     n_jobs=-1,  # Use all CPU cores
#     verbose=2
# )

# # Run Grid Search on original data (X, y)
# grid_search.fit(X, y)

# # Get the best model
# best_model = grid_search.best_estimator_
# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"Best ROC AUC: {grid_search.best_score_:.4f}")


# # Predict using the best model
# y_pred = best_model.predict(X_test)
# y_proba = best_model.predict_proba(X_test)[:, 1]

# # Evaluate metrics
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("ROC AUC:", roc_auc_score(y_test, y_proba))
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))


## Updated Confusion Matrix:
```
cm = [[605815   9834]
      [    72 615578]]
```

### Explanation:
- **True Positives (TP)**: The number of instances correctly predicted as **Class 1** (attack).
  - **615578** (bottom-right cell)
  
- **False Positives (FP)**: The number of instances incorrectly predicted as **Class 1** when they are actually **Class 0** (normal).
  - **9834** (top-right cell)

- **True Negatives (TN)**: The number of instances correctly predicted as **Class 0** (normal).
  - **605815** (top-left cell)

- **False Negatives (FN)**: The number of instances incorrectly predicted as **Class 0** when they are actually **Class 1**.
  - **72** (bottom-left cell)

### Now, let's compute some key metrics:

1. **Accuracy**: The overall percentage of correct predictions.
   \[
   \text{Accuracy} = \frac{TP + TN}{TP + FP + TN + FN}
   \]
   \[
   \text{Accuracy} = \frac{615578 + 605815}{615578 + 9834 + 605815 + 72} = \frac{1227393}{1227399} \approx 0.99995
   \]
   **Accuracy ≈ 99.995%**

2. **Precision for Class 1 (attack)**: The percentage of predicted positive instances (Class 1) that are actually positive.
   \[
   \text{Precision (Class 1)} = \frac{TP}{TP + FP} = \frac{615578}{615578 + 9834} \approx 0.983
   \]
   **Precision (Class 1) ≈ 98.3%**

3. **Recall for Class 1 (attack)**: The percentage of actual positive instances (Class 1) that were correctly identified.
   \[
   \text{Recall (Class 1)} = \frac{TP}{TP + FN} = \frac{615578}{615578 + 72} \approx 0.99988
   \]
   **Recall (Class 1) ≈ 99.99%**

4. **F1 Score for Class 1**: The harmonic mean of precision and recall.
   \[
   \text{F1 Score (Class 1)} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \approx 2 \times \frac{0.983 \times 0.99988}{0.983 + 0.99988} \approx 0.991
   \]
   **F1 Score (Class 1) ≈ 99.1%**

5. **Precision for Class 0 (normal)**: The percentage of predicted negative instances (Class 0) that are actually negative.
   \[
   \text{Precision (Class 0)} = \frac{TN}{TN + FN} = \frac{605815}{605815 + 72} \approx 0.99988
   \]
   **Precision (Class 0) ≈ 99.99%**

6. **Recall for Class 0 (normal)**: The percentage of actual negative instances (Class 0) that were correctly identified.
   \[
   \text{Recall (Class 0)} = \frac{TN}{TN + FP} = \frac{605815}{605815 + 9834} \approx 0.984
   \]
   **Recall (Class 0) ≈ 98.4%**

7. **F1 Score for Class 0**: The harmonic mean of precision and recall for Class 0.
   \[
   \text{F1 Score (Class 0)} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \approx 2 \times \frac{0.99988 \times 0.984}{0.99988 + 0.984} \approx 0.992
   \]
   **F1 Score (Class 0) ≈ 99.2%**

### Conclusion:
- **Accuracy**: 99.995%, which suggests that the model is correct nearly 100% of the time.
- **For Class 1 (attack)**: The model performs very well with a high **precision** (98.3%) and **recall** (99.99%), making it excellent at detecting **attacks**.
- **For Class 0 (normal)**: The model also performs excellently for **normal** instances with a **precision** of 99.99% and **recall** of 98.4%.

The model appears to be performing very well overall, especially for both classes. The **attack class** (Class 1) has very few false negatives, and the **normal class** (Class 0) has very few false positives. However, you may want to look into **class imbalance**, since the numbers of instances of Class 1 and Class 0 may be quite different. But for this case, the model is performing well in distinguishing both classes.

## Hyperparameter Tuning using GridSearchCV

## Algorithm RF

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Load data
df = df_9.copy()

# 2. Split data
X = df.drop(['Flag', 'CAN ID'], axis=1)  # Remove the leakage feature
y = df['Flag']

# Stratified split to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,
    random_state=42
)

# 3. Initialize Random Forest with class weighting
rf = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  # Handles imbalance
    max_depth=10,             # Prevent overfitting
    n_jobs=-1,                # Use all cores
    random_state=42
)

# 4. Train model
rf.fit(X_train, y_train)

# 5. Evaluate
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:,1]

# Evaluate model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

print(f"\nROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# 6. Feature Importance
features = X.columns
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

# Create DataFrame
fi_df = pd.DataFrame({'Feature': features, 'Importance': importances, 'Std': std})
fi_df = fi_df.sort_values('Importance', ascending=False)

# Plot
plt.figure(figsize=(10,6))
plt.title("Feature Importances")
plt.bar(fi_df['Feature'], fi_df['Importance'], yerr=fi_df['Std'])
plt.xticks(rotation=45)
plt.show()

## Class Weight Tuning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt

# Load data
df = df_9.copy()


# 2. Split data
X = df.drop(['Timestamp', 'Flag', 'CAN ID'], axis=1)  # Remove the leakage feature
y = df['Flag']

# Stratified split to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,
    random_state=42
)

# 3. Manually calculate class weights based on the exact class ratio (~5.24:1)
# ratio = 3_078_250 / 587_521  # approximately 5.24
ratio = 10
class_weights = {0: 1, 1: ratio}  # Assign higher weight to minority class

# 4. Initialize Random Forest with manually calculated class weights
rf = RandomForestClassifier(
    n_estimators=100,
    class_weight=class_weights,  # using manual weights instead of 'balanced'
    max_depth=10,                # Prevent overfitting
    n_jobs=-1,                   # Use all available cores
    random_state=42
)

# 5. Train the model
rf.fit(X_train, y_train)

# 6. Evaluate the model
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# 7. Feature Importance
features = X.columns
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

# Create DataFrame for feature importances
fi_df = pd.DataFrame({'Feature': features, 'Importance': importances, 'Std': std})
fi_df = fi_df.sort_values('Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(fi_df['Feature'], fi_df['Importance'], yerr=fi_df['Std'])
plt.xticks(rotation=45)
plt.show()


## Threshold Adjustment

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, precision_recall_curve
import matplotlib.pyplot as plt

# Load data
df = df_9.copy()


X = df.drop(['Timestamp', 'Flag', 'CAN ID'], axis=1)  # Remove the leakage feature
y = df['Flag']

# 3. Split data with stratification to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

# 4. Initialize XGBoost classifier with built-in imbalance handling
model = xgb.XGBClassifier(
    scale_pos_weight=5.24,      # Directly accounts for the class imbalance ratio
    objective='binary:logistic',
    eval_metric='aucpr',        # Optimize for precision-recall AUC
    use_label_encoder=False,    # Suppress a warning regarding label encoding
    random_state=42
)

# 5. Train the model
model.fit(X_train, y_train)

# 6. Get predicted probabilities for the positive class
y_proba = model.predict_proba(X_test)[:, 1]

# 7. Threshold Adjustment: Optimize decision threshold using precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Compute F1 scores for each threshold; note that thresholds array is one element shorter than precision/recall
f1_scores = 2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-8)  # avoid division by zero
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print(f"Optimal Threshold (based on maximum F1): {best_threshold:.4f}")

# Use the optimal threshold to make final predictions
y_pred = (y_proba >= best_threshold).astype(int)

# 8. Evaluate the model with the adjusted threshold
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

# 9. Feature Importance Plot
plt.figure(figsize=(10, 6))
plt.title("Feature Importances (XGBoost)")
xgb.plot_importance(model, ax=plt.gca())
plt.show()


## Hyperparameter Tuning:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)

print("Best parameters found:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

# Evaluate the tuned model
y_pred_tuned = best_rf.predict(X_test)
print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred_tuned))





2. **Advanced Handling of CAN ID**:

In [ ]:
   # Create interaction features between CAN ID and data bytes
   df['CAN_ID_Interaction'] = df['CAN ID'].astype(str) + '_' + df['DATA0'].astype(str)

3. **Dimensionality Reduction**:

In [ ]:
 from sklearn.decomposition import PCA
   
   pca = PCA(n_components=5)
   X_pca = pca.fit_transform(X)